In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'nonsand'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=nonsand'

In [5]:
output_file_path = '../data/labels/gt-bands.xlsx'

In [6]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ns-1,25.230858,83.021355,greenveg,2022-02-19,Vegetated Fields,Author,India,Off Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ns-2,25.246509,83.025825,bare,2022-02-19,Bare fields,Author,India,Off Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ns-4,22.573816,88.349404,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ns-6,19.752572,85.702946,whitewater,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ns-9,-23.855739,35.552899,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
#drop any columns that have 'Unnamed:' in the column name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport
1,ns-1,25.230858,83.021355,greenveg,2022-02-19,Vegetated Fields,Author,India,Off Ganga
2,ns-2,25.246509,83.025825,bare,2022-02-19,Bare fields,Author,India,Off Ganga
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga
4,ns-4,22.573816,88.349404,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport
6,ns-6,19.752572,85.702946,whitewater,2020-09-08,Off Puri,Author,India,Puri beach
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach
9,ns-9,-23.855739,35.552899,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN


In [8]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [9]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes'],
      dtype='object')

In [10]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [11]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport
1,ns-1,25.230858,83.021355,greenveg,2022-02-19,Vegetated Fields,Author,India,Off Ganga
2,ns-2,25.246509,83.025825,bare,2022-02-19,Bare fields,Author,India,Off Ganga
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga
4,ns-4,22.573816,88.349404,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport
6,ns-6,19.752572,85.702946,whitewater,2020-09-08,Off Puri,Author,India,Puri beach
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach
9,ns-9,-23.855739,35.552899,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN


In [12]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm
import file_management as fm

In [15]:
INDEX = 22

In [72]:
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [83]:
#Uncomment to change the date of the image
output['Date'].iloc[INDEX] = '2021-07-11'
# output['ID'].iloc[INDEX] = 'ns-49'

In [85]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='median', sampling_buffer_m=sampling_buffer_m, 
                                 max_search_window_months=3, median_samples=5,roi_buffer_m=5000, obia=True)

Index:  49  ID:  ns-49 Class:  bare  Site:  SFO tarmac
Search window from 2020-05-13 to 2020-11-13


## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function.
It will also overwrite just the sheet if the same exists, but will preserve the other sheets in the book

In [86]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
# fm.write_to_excel(output, output_file_path, sheet_name)
with pd.ExcelWriter(output_file_path,engine="openpyxl",mode="a",if_sheet_exists="replace") as writer:
    output.to_excel(writer,sheet_name=sheet_name,index=False)
output.iloc[INDEX-1]

Original marker accepted
Kept Observation


ID                        ns-49
Latitude              37.627765
Longitude           -122.381046
Class                      bare
Date                 2020-08-13
Site                 SFO tarmac
Source                      NaN
Country                     USA
Notes                       NaN
B2_mean             2774.731689
B3_mean             2975.707275
B4_mean             3003.268311
B8_mean             2925.731689
B8A_mean            2871.987793
B11_mean            3407.524414
B12_mean            3736.890137
VV_mean              -15.805871
VH_mean               -24.73904
mTGSI_mean             0.067513
BSI_mean               0.058926
NDWI_mean               0.00815
keep                       True
location_tweaked          False
class_code                    5
Name: 49, dtype: object

## Final Output

In [84]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2_mean,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,ns-0,25.449886,82.858051,bare,2022-10-03,Airport,Author,India,Varanasi Airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
1,ns-1,25.230858,83.021355,greenveg,2022-02-19,Vegetated Fields,Author,India,Off Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
2,ns-2,25.246509,83.025825,bare,2022-02-19,Bare fields,Author,India,Off Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
3,ns-3,25.208222,82.980644,water,2022-02-19,River water,Author,India,Ganga,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
4,ns-4,22.573816,88.349404,bare,2022-01-02,"Urban scene, kolkata",Author,India,Off Hoogly,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
5,ns-5,22.642989,88.439406,bare,2020-05-11,Airport building,Author,India,Kolkata airport,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
6,ns-6,19.752572,85.702946,whitewater,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
7,ns-7,19.744671,85.694326,water,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
8,ns-8,19.762033,85.703707,greenveg,2020-09-08,Off Puri,Author,India,Puri beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
9,ns-9,-23.855739,35.552899,whitewater,2020-01-02,Off Tofo Beach,Author,Mozambique,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
